# CPSC 4810 TEAM PROJECT 
### Member: 
- An Nguyen
- Andy Nguyen
- Dawn Dang - 100413772
- Huan Le Van

# TABLE OF CONTENTS
1. PROJECT DESCRIPTION
2. THE DATA
    1. Supplemental Data
3. BACKGROUND RESEARCH
4. READ IN THE DATA
    1. Reading .csv files
    2. Reading .txt files
        1. Data Selection
5. GETTING TO KNOW THE DATA
6. DATA PREPARATION
    1. Jobs postings
    2. Company details

## PROJECT DESCRIPTION

Our project focuses on analyzing job postings on LinkedIn in the USA last year, using a combination of datasets to ensure we cover all the necessary information. We prioritize data acquisition, preparation, cleaning, and aggregation to create a top-notch dataset for exploratory data analysis (EDA). Through EDA, we aim to discover important insights into the job market, like popular skills, common job titles, where opportunities are located, and trends in different industries. By providing clear insights, our project aims to help decision-making and improve the job market.

## THE DATA

Every day, LinkedIn serves as a go-to platform for thousands of companies and individuals seeking talent or career opportunities. This dataset offers an extensive collection of over 33,000 job postings spanning two distinct days, months apart. Each job listing is rich in detail, encompassing 27 key attributes such as job title, description, salary, location, application URL, and work arrangements (including remote or contract positions). Additionally, supplemental files provide insights into associated benefits, required skills, and relevant industries. Linked to the majority of job postings are corresponding company profiles, detailed in a separate CSV file, featuring company descriptions, headquarters location, employee count, and follower metrics.
Our main datasets:
- companies.csv - Details about hiring companies
- job_postings.csv - Details about the jobs 

In [3]:
#Import library
import pandas as pd
import numpy as np


#### Jobs_postings

In [4]:
#load data
jobs_posting = pd.read_csv('./data/job_postings.csv')
jobs_posting.head(5)

,job_id,company_id,title,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped
0,3757940104,553718.0,Hearing Care Provider,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",NaN,...,NaN,Entry level,NaN,1.700000e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",NaN,...,NaN,NaN,NaN,1.700000e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420
2,3757938019,474443.0,"Manager, Engineering",NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",NaN,...,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.700000e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644
3,3757938018,18213359.0,Cook,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",NaN,...,NaN,Entry level,NaN,1.700000e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),275834.0,NaN,205956.0,YEARLY,Full-time,United States,NaN,...,NaN,Mid-Senior level,NaN,1.700000e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346


#### company_detail

In [ ]:
merged_data = pd.merge(jobs, comp, on='company_id', how='left')
comp_size = merged_data[['company_id', 'company_size']]
comp_size.to_csv('jobs_with_company_size.csv', index=False)

## Supplemental Data

Our project also utilise supplemental files from the same source; that offer enriched insights into the job market. These additional datasets provide valuable information on benefits, skills, industries, and company profiles associated with each job posting. By exploring these supplementary datasets, users gain deeper insights into job market dynamics, enabling informed decision-making and strategic planning in talent acquisition and employment.

Supplement datasets:
- **company_industries.csv** : Details on hiring compannies' registered industry names including company IDs.
- **industries.csv** and **jobs_industries.csv**: Lists of industry IDs, matching job IDs, and industry names.
- **company_specialities.csv**: Details on hiring companies special field.
- **employee_counts.csv**: Provide information about hiring companies' number of current employees, its followers and well as the time said data were collected.
- **job_skills.csv** and **skills.csv**: Lists of the key required skills for each job, along with matching job IDs and skill abbreviations.
- **benefit.csv**: contains lists of benefits offered, each matched with corresponding job IDs. 
- **Salaries.csv**: provides information on the compensation range for each job ID, including pay period, currency, and compensation type.

## Background Research

Our research centers on analyzing job advertisements on LinkedIn within the United States in 2023. We aim to understand job posting trends, companies' profiles, and job requirements and compensations. Given the complexity of the datasets, we must clean, map, combine, or drop datasets to ensure accuracy.

Key areas of focus include:

- Job Ads Posted on LinkedIn in the US: Examining trends and geographical variations across 50 states.
- Companies Posting Jobs: Analyzing company profiles, including size and industry.
- Job Requirements and Compensation: Investigating diverse skill sets and salary structures.

Our analysis acknowledges variability across states and industries, guiding us to consolidate datasets for effective exploration of correlations and patterns.

## Data Preparation

### Mapping company sizes
We generate a mapping file to correlate company size levels (1-7) with real company sizes on LinkedIn, aiming to explore how company size influences offered skills and salaries in job postings.

In [13]:
merged_data = pd.merge(jobs, comp, on='company_id', how='left')
comp_size = merged_data[['company_id', 'company_size']]
comp_size.to_csv('jobs_with_company_size.csv', index=False)

NameError: name 'jobs' is not defined

### Convert number to datetime format
Data store the date and time the posting was posted, expired but in epoch format so we need to convert to readable date time format

In [12]:
jobs_posting['new_original_listed_time'] = pd.to_datetime(jobs_posting['original_listed_time'], unit='ms')
jobs_posting['new_closed_time'] = pd.to_datetime(jobs_posting['closed_time'], unit='ms')
jobs_posting['new_listed_time'] = pd.to_datetime(jobs_posting['listed_time'], unit='ms')

C:\Users\hoang\AppData\Local\Temp\ipykernel_8912\3902175047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_posting['new_original_listed_time'] = pd.to_datetime(jobs_posting['original_listed_time'], unit='ms')
C:\Users\hoang\AppData\Local\Temp\ipykernel_8912\3902175047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_posting['new_closed_time'] = pd.to_datetime(jobs_posting['closed_time'], unit='ms')
C:\Users\hoang\AppData\Local\Temp\ipykernel_8912\3902175047.py:3: SettingWithCopyWarning: 
A

In [ ]:
# hello do you see me? lol

### Calculate yearly average salary
avarage = median salary
or = (max + min) /2

In [95]:
# Fill NaN values in 'med_salary' with the average of 'max_salary' and 'min_salary'
jobs_posting['avg_salary'] = jobs_posting['med_salary'].fillna((jobs_posting['max_salary'] + jobs_posting['min_salary']) / 2)

# Change all salary to yearly salary unit
jobs_posting['yearly_avg_salary'] = jobs_posting['avg_salary']
multiplier_map = {'YEARLY': 1,
                  'HOURLY': 40 * 52.2,
                  'MONTHLY': 12,
                  'WEEKLY': 52.2,
                  'ONCE': 1}
jobs_posting['yearly_avg_salary'] *= jobs_posting['pay_period'].map(multiplier_map)
jobs_posting.head(5)

,job_id,company_id,title,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,avg_salary,yearly_avg_salary
0,3757940104,553718.0,Hearing Care Provider,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",NaN,...,NaN,1.700000e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,5250.00,63000.00
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",NaN,...,NaN,1.700000e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420,NaN,NaN
2,3757938019,474443.0,"Manager, Engineering",NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",NaN,...,Bachelor's Degree in Mechanical Engineering pr...,1.700000e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644,NaN,NaN
3,3757938018,18213359.0,Cook,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",NaN,...,NaN,1.700000e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,22.27,46499.76
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),275834.0,NaN,205956.0,YEARLY,Full-time,United States,NaN,...,NaN,1.700000e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346,240895.00,240895.00


In [97]:
jobs_posting[jobs_posting['yearly_avg_salary'].notnull()]

,job_id,company_id,title,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,avg_salary,yearly_avg_salary
0,3757940104,553718.0,Hearing Care Provider,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",NaN,...,NaN,1.700000e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,5250.00,63000.00
3,3757938018,18213359.0,Cook,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",NaN,...,NaN,1.700000e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,22.27,46499.76
4,3757937095,437225.0,Principal Cloud Security Architect (Remote),275834.0,NaN,205956.0,YEARLY,Full-time,United States,NaN,...,NaN,1.700000e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346,240895.00,240895.00
8,3757936097,18213359.0,Dishwasher,NaN,19.30,NaN,HOURLY,Full-time,"Aliso Viejo, CA",NaN,...,NaN,1.700000e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699089324,19.30,40298.40
11,3757935381,19181907.0,Insights Analyst - Auto Industry,64000.0,NaN,58000.0,YEARLY,Full-time,United States,NaN,...,NaN,1.700000e+12,escalent.hrmdirect.com,0,FULL_TIME,USD,BASE_SALARY,1699134111,61000.00,61000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33231,1198636648,NaN,Mental Health Practitioners,NaN,100000.00,NaN,YEARLY,Full-time,"Minneapolis, MN",NaN,...,NaN,1.700000e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,1699063051,100000.00,100000.00
33233,1029078768,61469.0,Registered Nurse (RN) Vaccinator,50.0,NaN,50.0,HOURLY,Part-time,"Muskegon, MI",NaN,...,NaN,1.690000e+12,NaN,0,PART_TIME,USD,BASE_SALARY,1,50.00,104400.00
33236,903408693,3894635.0,Office Associate,42000.0,NaN,37000.0,YEARLY,Full-time,"Albany, GA",5.0,...,NaN,1.690000e+12,NaN,1,FULL_TIME,USD,BASE_SALARY,1,39500.00,39500.00
33244,85008768,NaN,Licensed Insurance Agent,52000.0,NaN,45760.0,YEARLY,Full-time,"Chico, CA",NaN,...,NaN,1.690000e+12,NaN,1,FULL_TIME,USD,BASE_SALARY,1,48880.00,48880.00


# Remove NA row for missing experience level

In [5]:
jobs_posting.columns.tolist()

['job_id',
 'company_id',
 'title',
 'max_salary',
 'med_salary',
 'min_salary',
 'pay_period',
 'formatted_work_type',
 'location',
 'applies',
 'original_listed_time',
 'remote_allowed',
 'views',
 'job_posting_url',
 'application_url',
 'application_type',
 'expiry',
 'closed_time',
 'formatted_experience_level',
 'skills_desc',
 'listed_time',
 'posting_domain',
 'sponsored',
 'work_type',
 'currency',
 'compensation_type',
 'scraped',
 'avg_salary',
 'yearly_avg_salary']

In [6]:
jobs_posting['formatted_experience_level'].value_counts()

formatted_experience_level
Mid-Senior level    11258
Entry level          7969
Associate            2793
Director             1354
Internship            387
Executive             304
Name: count, dtype: int64

In [7]:
jobs_posting['formatted_experience_level'].isna().sum()

9181

In [8]:
jobs_posting['formatted_experience_level'].notna().sum()

24065

In [11]:
jobs_posting = jobs_posting[jobs_posting['formatted_experience_level'].notnull()]
jobs_posting['formatted_experience_level'].isna().sum()

0

#### Location 

The Location column in the dataset include 4 format of infomation:
- Country (United States)
- City + Shortname of state (Ex: New York, NY ..)
- City + State + Country (Ex: St Clair Shores, Michigan, United States ..)
- Metropolitan Area (Ex: New York City Metropolitan Area ..)

In the scope of this analysis, we just deal with the 2nd & 3rd types, which is to process the state infomation



In [9]:
jobs_posting.groupby('location').size().reset_index(name = 'counts').sort_values(by = 'counts', ascending=False).head(15)

,location,counts
4203,United States,2341
2925,"New York, NY",818
742,"Chicago, IL",534
1920,"Houston, TX",444
972,"Dallas, TX",383
169,"Atlanta, GA",382
2393,"Los Angeles, CA",366
198,"Austin, TX",347
421,"Boston, MA",324
3645,"San Francisco, CA",289


External dataset __abbr-name.json__ includes infomation of shortname mapping to name of state

In [50]:
df_states = pd.read_json('./data/abbr-name.json', orient='index')
df_states.reset_index(inplace=True)
df_states.columns = ['state_short_name', 'state']
df_states.head(5)

,state_short_name,state
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [51]:
df_states[df_states['state'] == 'New York']

,state_short_name,state
32,NY,New York


Filter list of locations (City, State) from the job_posting dataset:

In [22]:
df_location  = pd.DataFrame(jobs_posting[(jobs_posting['location'].str.contains(','))]['location'].unique())
df_location.columns  = ['Location']
df_location.head(5)

,Location
0,"Little River, SC"
1,"Beaver Dam, WI"
2,"Bessemer, AL"
3,"Aliso Viejo, CA"
4,"Daytona Beach, FL"


In [43]:
df_location[['city','state','country']] = df_location['Location'].str.split(', ', expand=True)
df_location

,Location,city,state,country
0,"Little River, SC",Little River,SC,None
1,"Beaver Dam, WI",Beaver Dam,WI,None
2,"Bessemer, AL",Bessemer,AL,None
3,"Aliso Viejo, CA",Aliso Viejo,CA,None
4,"Daytona Beach, FL",Daytona Beach,FL,None
...,...,...,...,...
4486,"Thomson, GA",Thomson,GA,None
4487,"Pendleton, OR",Pendleton,OR,None
4488,"Arlington, WA",Arlington,WA,None
4489,"Bohemia, NY",Bohemia,NY,None


Now we join with state list (__abbr-name.json__) to get statename

In [86]:
df_location_merge = pd.merge(df_location, df_states, left_on='state', right_on='state_short_name', how='left')
df_location_merge

,Location,city,state_x,country,state_short_name,state_y
0,"Little River, SC",Little River,SC,None,SC,South Carolina
1,"Beaver Dam, WI",Beaver Dam,WI,None,WI,Wisconsin
2,"Bessemer, AL",Bessemer,AL,None,AL,Alabama
3,"Aliso Viejo, CA",Aliso Viejo,CA,None,CA,California
4,"Daytona Beach, FL",Daytona Beach,FL,None,FL,Florida
...,...,...,...,...,...,...
4486,"Thomson, GA",Thomson,GA,None,GA,Georgia
4487,"Pendleton, OR",Pendleton,OR,None,OR,Oregon
4488,"Arlington, WA",Arlington,WA,None,WA,Washington
4489,"Bohemia, NY",Bohemia,NY,None,NY,New York


However, when filtering the location with column country is not null, we can see that the state_x column is already in full state name. So we just update the state_y colums = state_x columns for these rows

In [87]:
df_location_merge[df_location_merge['country'].notnull()].head(5)

,Location,city,state_x,country,state_short_name,state_y
9,"St Clair Shores, Michigan, United States",St Clair Shores,Michigan,United States,NaN,NaN
175,"Wilmington, Delaware, United States",Wilmington,Delaware,United States,NaN,NaN
244,"Denver, Colorado, United States",Denver,Colorado,United States,NaN,NaN
273,"Fremont, California, United States",Fremont,California,United States,NaN,NaN
512,"Los Angeles, California, United States",Los Angeles,California,United States,NaN,NaN


In [88]:
df_location_merge.loc[df_location_merge['country'].notnull(), 'state_y'] = df_location_merge['state_x']

In [89]:
df_location_merge[df_location_merge['country'].notnull()].head(5)

,Location,city,state_x,country,state_short_name,state_y
9,"St Clair Shores, Michigan, United States",St Clair Shores,Michigan,United States,NaN,Michigan
175,"Wilmington, Delaware, United States",Wilmington,Delaware,United States,NaN,Delaware
244,"Denver, Colorado, United States",Denver,Colorado,United States,NaN,Colorado
273,"Fremont, California, United States",Fremont,California,United States,NaN,California
512,"Los Angeles, California, United States",Los Angeles,California,United States,NaN,California


In [90]:
# Drop redundant columns
df_location_merge.drop(columns=['state_x','country','state_short_name'], inplace=True)

# Rename the columns for clarity
df_location_merge.rename(columns={'state_y': 'state'}, inplace=True)

In [91]:
#drop NA:
df_location_merge.dropna(subset = ['state'])

,Location,city,state
0,"Little River, SC",Little River,South Carolina
1,"Beaver Dam, WI",Beaver Dam,Wisconsin
2,"Bessemer, AL",Bessemer,Alabama
3,"Aliso Viejo, CA",Aliso Viejo,California
4,"Daytona Beach, FL",Daytona Beach,Florida
...,...,...,...
4486,"Thomson, GA",Thomson,Georgia
4487,"Pendleton, OR",Pendleton,Oregon
4488,"Arlington, WA",Arlington,Washington
4489,"Bohemia, NY",Bohemia,New York


Now we finally join with the original dataset to get the state:


In [94]:
jobs_posting_lcn = pd.merge(jobs_posting, df_location_merge, left_on='location', right_on='Location', how='left')
jobs_posting_lcn.dropna(subset = ['state'])

,job_id,company_id,title,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,Location,city,state
0,3757940104,553718.0,Hearing Care Provider,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",NaN,...,1.700000e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,"Little River, SC",Little River,South Carolina
1,3757940025,2192142.0,Shipping & Receiving Associate 2nd shift (Beav...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",NaN,...,1.700000e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420,"Beaver Dam, WI",Beaver Dam,Wisconsin
2,3757938019,474443.0,"Manager, Engineering",NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",NaN,...,1.700000e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644,"Bessemer, AL",Bessemer,Alabama
3,3757938018,18213359.0,Cook,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",NaN,...,1.700000e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,"Aliso Viejo, CA",Aliso Viejo,California
6,3757937004,10515052.0,Auto Body Techncian,NaN,NaN,NaN,NaN,Full-time,"Daytona Beach, FL",NaN,...,1.700000e+12,boydgroup.wd1.myworkdayjobs.com,0,FULL_TIME,NaN,NaN,1699089473,"Daytona Beach, FL",Daytona Beach,Florida
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33240,133196985,1089558.0,Model Risk Auditor,NaN,NaN,NaN,NaN,Contract,"New York, NY",1.0,...,1.690000e+12,NaN,0,CONTRACT,NaN,NaN,1,"New York, NY",New York,New York
33241,133114754,77766802.0,Sales Manager,NaN,NaN,NaN,NaN,Full-time,"Santa Clarita, CA",NaN,...,1.690000e+12,NaN,0,FULL_TIME,NaN,NaN,1,"Santa Clarita, CA",Santa Clarita,California
33242,108965123,NaN,Office Administrative Assistant,NaN,NaN,NaN,NaN,Full-time,"New York, NY",2.0,...,1.700000e+12,NaN,0,FULL_TIME,NaN,NaN,1699044401,"New York, NY",New York,New York
33244,85008768,NaN,Licensed Insurance Agent,52000.0,NaN,45760.0,YEARLY,Full-time,"Chico, CA",NaN,...,1.690000e+12,NaN,1,FULL_TIME,USD,BASE_SALARY,1,"Chico, CA",Chico,California
